## Code RAG (VectorDB) Creation
This notebook uses FAISS as the vectorDB to store and retreive similar questions from the example databses to provide few shot learning examples

In [1]:
# !pip install faiss-cpu

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import jupyter_black

jupyter_black.load()

In [4]:
import pandas as pd
import os
import json
import boto3

from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS

from config import (
    FAISS_DICT_EXAMPLES_PATH,
    FAISS_INDEX_DIR,
    EMBED_MODEL_ID,
)

## Load Golden Examples from Json

In [5]:
# Load Dictionary of examples
with open(FAISS_DICT_EXAMPLES_PATH, "r") as fp:
    few_shot_examples = json.load(fp)

# Show the examples
for key, val in few_shot_examples.items():
    print(f"\nQuestion: {key}\n")
    print(f"SQL: {val}")
    print("-" * 30)


Question: Select all patients

SQL: SELECT desynpuf_id FROM rwdex_raw_synpuf.beneficiary_summary;
------------------------------

Question: Select all female inpatients diagnosed with diabetes of code list X.

SQL: SELECT DISTINCT ic.desynpuf_id FROM rwdex_raw_synpuf.inpatient_claims ic INNER JOIN rwdex_raw_synpuf.beneficiary_summary bs ON ic.desynpuf_id = bs.desynpuf_id WHERE bs.bene_sex_ident_cd = 2 AND ic.admtng_icd9_dgns_cd IN (CODE_LIST_X);
------------------------------

Question: Find total number of male, hispanic outpatients living in Georgia.

SQL: SELECT COUNT(DISTINCT oc.desynpuf_id) AS num_patients FROM rwdex_raw_synpuf.outpatient_claims oc INNER JOIN rwdex_raw_synpuf.beneficiary_summary bs ON oc.desynpuf_id = bs.desynpuf_id WHERE bs.bene_sex_ident_cd = 2 AND bs.bene_race_cd = 5 AND bs.sp_state_code = 11;
------------------------------


## Create question embeddings and store to FAISS

In [6]:
# Create embeddings model
bedrock_runtime = boto3.client("bedrock-runtime")
llm_embed = BedrockEmbeddings(client=bedrock_runtime, model_id=EMBED_MODEL_ID)

In [10]:
questions = list(few_shot_examples.keys())

In [11]:
# Create embeddings for the sample questions and save
db = FAISS.from_texts(questions, llm_embed)
db.save_local(FAISS_INDEX_DIR)